In [7]:
from glob import glob
from os import mkdir
from os.path import join, basename

import pandas as pd
import numpy as np

In [9]:
common_path = "/Volumes/MMIS-Saraiv/Datasets/Izmir/EC/as_given_denoised_2s-epoched"
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Izmir/EC/128Hz_denoised_2s-epoched"
original_sf = 500.0
new_sf = 128.0

In [10]:
all_subjects = glob(join(common_path, '*'))
all_subjects

In [13]:
for subject_path in all_subjects:
    subject = basename(subject_path).split('-')[0].split(' ')[1]
    print("Subject Code", subject)
    all_files = glob(join(subject_path, '*.txt'))
    
    out_subject_path = join(out_common_path, basename(subject_path))
    mkdir(out_subject_path)

    # go by epoch
    for file in all_files:
        data = pd.read_csv(file, sep='\t', dtype=float)  # time x channels
        resampled_data = []
        # go by channel (column)
        for i in range(data.shape[1]):
            channel = np.array(data.iloc[:, i].values)
            # resample with numpy
            channel = np.interp(np.arange(0, len(channel), original_sf/new_sf), np.arange(0, len(channel)), channel)
            # put it back in the dataframe
            resampled_data.append(channel)
        resampled_data = pd.DataFrame(np.array(resampled_data).T)
        # save the resampled data
        new_file = join(out_subject_path, basename(file))
        resampled_data.to_csv(new_file, sep='\t', index=False, header=False)